In [162]:
# import needed
import pandas as pd # data processing
import pandas_profiling as pp
import numpy as np # linear algebra
import pprint #indet json 
import requests #make http requests
from qwikidata.sparql  import return_sparql_query_results #return sparql results

In [163]:
#open datasets
df_movie_title = pd.read_csv('savelist.csv')
#df_movie_id = pd.read_csv('savelist2.csv')
df_serie_title = pd.read_csv('series_director.csv')
df_serie_id = pd.read_csv('series.csv')

In [164]:
df_first = pd.read_csv('0-6000.csv')
df_first = df_first.loc[:, ~df_first.columns.str.contains('^Unnamed')]
df_first.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
df_first.head(5)
#df_first.info()
#print(len(df_first))

,title,distributor,id,rating score
0,Sankofa,no_data,tt0108041,8.2/10
1,The Starling,Netflix,tt5164438,4.2/10
2,Confessions of an Invisible Girl,Netflix,tt15204288,no_data
3,Intrusion,no_data,tt5563324,4.5/10
4,Avvai Shanmughi,no_data,tt0139872,no_data


In [165]:
df_last = pd.read_csv('9000_end.csv')
df_last = df_last.loc[:, ~df_last.columns.str.contains('^Unnamed')]
df_last.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
df_last.head(5)

#df_last.info()
#print(len(df_last))

,title,distributor,id,rating score
0,Unforgivable,no_data,tt13003692,no_data
1,Sucker Punch,iTunes,tt0978764,4.2/10
2,Sherni,no_data,tt0187060,no_data
3,Shaandaar,Star Studios,tt4007558,3.9/10
4,LOL,Netflix,tt1592873,3.7/10


In [166]:
df_but_last = pd.read_csv('6000-9000.csv')
df_but_last = df_but_last.loc[:, ~df_but_last.columns.str.contains('^Unnamed')]
df_but_last.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
df_but_last.head(5)
#df_but_last.info()
#print(len(df_but_last))

,title,distributor,id,rating score
0,Slappy and the Stinkers,TriStar Pictures,tt0120213,3.3/10
1,Sky Racket,Sam Katzman,tt0140551,no_data
2,Sitting on the Moon,Republic Pictures,tt0028260,no_data
3,Sita,no_data,tt9665402,no_data
4,Sing Sing Nights,no_data,tt0027003,no_data


In [167]:
df_movie_id = pd.concat([df_first, df_but_last, df_last],axis=0, join="outer", sort=False)
df_movie_id.head(5)

,title,distributor,id,rating score
0,Sankofa,no_data,tt0108041,8.2/10
1,The Starling,Netflix,tt5164438,4.2/10
2,Confessions of an Invisible Girl,Netflix,tt15204288,no_data
3,Intrusion,no_data,tt5563324,4.5/10
4,Avvai Shanmughi,no_data,tt0139872,no_data


# take care of multiple distributors/ratings

In [168]:
# check
pd.concat(g for _, g in df_movie_id.groupby("title") if len(g) > 1)

,title,distributor,id,rating score
596,#Home,Amazon Prime Video,tt10534500,no_data
1950,#Home,Amazon Prime Video,tt10534500,no_data
565,1 Night in San Diego,no_data,tt9765226,no_data
1919,1 Night in San Diego,no_data,tt9765226,no_data
472,10 Endrathukulla,Star Studios,tt5128266,no_data
...,...,...,...,...
3782,Zubaan,no_data,tt3776484,no_data
4492,Zubaan,no_data,tt3776484,no_data
5018,Zubaan,no_data,tt3776484,no_data
5544,Zubaan,no_data,tt3776484,no_data


In [169]:
df_movie_id_copy = df_movie_id.drop_duplicates(keep='first')

In [170]:
# check
pd.concat(g for _, g in df_movie_id_copy.groupby("title") if len(g) > 1)

,title,distributor,id,rating score
1401,10 Things I Hate About You,Disney+,tt0147800,69%
1511,10 Things I Hate About You,Netflix,tt0147800,69%
1258,10 to Midnight,The Cannon Group,tt0085121,40%
2612,10 to Midnight,Netflix,tt0085121,40%
772,A Chinese Ghost Story,no_data,tt1861421,no_data
...,...,...,...,...
6998,Zandalee,Netflix,tt0101004,4/10
3784,Zoe,Netflix,tt6010628,4.3/10
5546,Zoe,Amazon Studios,tt6010628,4.3/10
3780,Zombieland,InterCom,tt1156398,7.4/10


In [171]:
# check
df_movie_id_copy.head(7)
df_movie_id_copy.query('title=="10 Things I Hate About You"')

,title,distributor,id,rating score
1401,10 Things I Hate About You,Disney+,tt0147800,69%
1511,10 Things I Hate About You,Netflix,tt0147800,69%


In [172]:
df_movie_id_toupled = df_movie_id_copy.groupby(['title', 'id']).agg(tuple).reset_index()
df_movie_id_toupled.head(7)

,title,id,distributor,rating score
0,#Alive,tt10620868,"(Netflix,)","(88%,)"
1,#Home,tt10534500,"(Amazon Prime Video,)","(no_data,)"
2,#Roxy,tt5646082,"(iTunes,)","(no_data,)"
3,#realityhigh,tt6119504,"(Netflix,)","(40%,)"
4,'76,tt2385126,"(YouTube,)","(no_data,)"
5,1 Chance 2 Dance,tt2388856,"(Netflix,)","(no_data,)"
6,1 Mile to You,tt2184233,"(Gravitas Ventures,)","(no_data,)"


In [173]:
# check
df_movie_id_toupled.query('title=="10 Things I Hate About You"')	

,title,id,distributor,rating score
13,10 Things I Hate About You,tt0147800,"(Disney+, Netflix)","(69%, 69%)"


In [174]:
df_movie_id_clean_copy = df_movie_id_toupled.copy() #estraiamo i valori singoli dalle tuple e manteniamo un vlore solo se sono duplicati i rotten rates
for n_index, title in df_movie_id_clean_copy['title'].iteritems():
    distributor = df_movie_id_clean_copy["distributor"][n_index]
    rating = df_movie_id_clean_copy["rating score"][n_index]
    
    if len(distributor)== 1:
        df_movie_id_clean_copy.at[n_index,'distributor']=distributor[0]

    if len(rating) > 1:
        
        for i in rating:
            rating_check = set()
            rating_check.add(i)
            
            if len(rating_check) <= 1:
                df_movie_id_clean_copy.at[n_index,'rating score']=rating[0]
                

    if len(rating) == 1:
        
        df_movie_id_clean_copy.at[n_index,'rating score']=rating[0]

In [175]:
df_movie_id_clean_copy

,title,id,distributor,rating score
0,#Alive,tt10620868,Netflix,88%
1,#Home,tt10534500,Amazon Prime Video,no_data
2,#Roxy,tt5646082,iTunes,no_data
3,#realityhigh,tt6119504,Netflix,40%
4,'76,tt2385126,YouTube,no_data
...,...,...,...,...
6817,Zubaan,tt3776484,no_data,no_data
6818,Æon Flux,tt0402022,Paramount Pictures,3.5/10
6819,Çarsi Pazar,tt4324242,Netflix,no_data
6820,Çok Filim Hareketler Bunlar,tt1640202,Netflix,no_data


In [176]:
# check
df_movie_id_clean_copy.query('title=="10 Things I Hate About You"')	

,title,id,distributor,rating score
13,10 Things I Hate About You,tt0147800,"(Disney+, Netflix)",69%


# prepare autor/autor gender dataframe

In [177]:
df_movie_title #autor/autor gender
df_movie_id_clean_copy #id/distributor/rottenscore

,title,id,distributor,rating score
0,#Alive,tt10620868,Netflix,88%
1,#Home,tt10534500,Amazon Prime Video,no_data
2,#Roxy,tt5646082,iTunes,no_data
3,#realityhigh,tt6119504,Netflix,40%
4,'76,tt2385126,YouTube,no_data
...,...,...,...,...
6817,Zubaan,tt3776484,no_data,no_data
6818,Æon Flux,tt0402022,Paramount Pictures,3.5/10
6819,Çarsi Pazar,tt4324242,Netflix,no_data
6820,Çok Filim Hareketler Bunlar,tt1640202,Netflix,no_data


In [178]:
df_movie_title.head(7)

,Unnamed: 0,title,director,director gender
0,0,Sankofa,Haile Gerima,male
1,1,The Starling,Theodore Melfi,male
2,2,Confessions of an Invisible Girl,no_data,no_data
3,3,Intrusion,no_data,no_data
4,4,Avvai Shanmughi,K. S. Ravikumar,male
5,5,Jeans,Nicolette Krebitz,female
6,6,Minsara Kanavu,Rajiv Menon,male


In [179]:
df_movie_title = df_movie_title.loc[:, ~df_movie_title.columns.str.contains('^Unnamed')]
df_movie_title.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
df_movie_title.head(7)

,title,director,director gender
0,Sankofa,Haile Gerima,male
1,The Starling,Theodore Melfi,male
2,Confessions of an Invisible Girl,no_data,no_data
3,Intrusion,no_data,no_data
4,Avvai Shanmughi,K. S. Ravikumar,male
5,Jeans,Nicolette Krebitz,female
6,Minsara Kanavu,Rajiv Menon,male


In [180]:
df_movie_title_copy = df_movie_title.drop_duplicates(keep='first')
len(df_movie_title)

6835

In [181]:
# check
print(len(df_movie_title_copy))
print(len(df_movie_id_clean_copy))


6688
6822


# merge 


In [182]:
df_movie_directors_copy = df_movie_title_copy.copy()
df_movie_directors_copy["distributor"] = " "
df_movie_directors_copy["id"] = " "
df_movie_directors_copy["rating score"] = " "
df_movie_directors_copy.head()

,title,director,director gender,distributor,id,rating score
0,Sankofa,Haile Gerima,male,,,
1,The Starling,Theodore Melfi,male,,,
2,Confessions of an Invisible Girl,no_data,no_data,,,
3,Intrusion,no_data,no_data,,,
4,Avvai Shanmughi,K. S. Ravikumar,male,,,


In [183]:
#check
df_movie_id_clean_copy.head(4)

,title,id,distributor,rating score
0,#Alive,tt10620868,Netflix,88%
1,#Home,tt10534500,Amazon Prime Video,no_data
2,#Roxy,tt5646082,iTunes,no_data
3,#realityhigh,tt6119504,Netflix,40%


In [184]:
#check
small_df_movie_directors_copy = df_movie_directors_copy[0:500]
small_df_movie_id_clean_copy = df_movie_id_clean_copy[0:500]

In [185]:
df_movie_directors_copy

,title,director,director gender,distributor,id,rating score
0,Sankofa,Haile Gerima,male,,,
1,The Starling,Theodore Melfi,male,,,
2,Confessions of an Invisible Girl,no_data,no_data,,,
3,Intrusion,no_data,no_data,,,
4,Avvai Shanmughi,K. S. Ravikumar,male,,,
...,...,...,...,...,...,...
6779,River,Shanice Mollen-Picard,female,,,
6780,Outpost,Chuck Jones,male,,,
6781,Harry Brown,Daniel Barber,male,,,
6821,Snow White,Michael Berz,male,,,


In [186]:
#check
small_df_movie_id_clean_copy

,title,id,distributor,rating score
0,#Alive,tt10620868,Netflix,88%
1,#Home,tt10534500,Amazon Prime Video,no_data
2,#Roxy,tt5646082,iTunes,no_data
3,#realityhigh,tt6119504,Netflix,40%
4,'76,tt2385126,YouTube,no_data
...,...,...,...,...
495,Arjun Reddy,tt7294534,no_data,no_data
496,Arjun: The Warrior Prince,tt2404027,Netflix,5.7/10
497,Arlo the Alligator Boy,tt13454122,Netflix,7.4/10
498,Armageddon,tt0120591,Netflix,38%


In [187]:
for index_n, title in df_movie_directors_copy['title'].iteritems():
   
    for index_id, title_id in df_movie_id_clean_copy['title'].iteritems():
        
        distributor = df_movie_id_clean_copy["distributor"][index_id]
        id = df_movie_id_clean_copy["id"][index_id]
        rating = df_movie_id_clean_copy["rating score"][index_id]
        
        if title == title_id:
            
            df_movie_directors_copy.at[index_n,'distributor']=distributor
            df_movie_directors_copy.at[index_n,'id']=id
            df_movie_directors_copy.at[index_n,'rating score']=rating 

In [203]:
#check
df_movie_directors_copy.head(7)

,title,director,director gender,distributor,id,rating score
0,Sankofa,Haile Gerima,male,no_data,tt0108041,8.2/10
1,The Starling,Theodore Melfi,male,Netflix,tt5164438,4.2/10
2,Confessions of an Invisible Girl,no_data,no_data,Netflix,tt15204288,no_data
3,Intrusion,no_data,no_data,no_data,tt5563324,4.5/10
4,Avvai Shanmughi,K. S. Ravikumar,male,no_data,tt0139872,no_data
5,Jeans,Nicolette Krebitz,female,no_data,tt0279885,no_data
6,Minsara Kanavu,Rajiv Menon,male,AVM Productions,tt0282003,no_data


In [189]:
# check
df_movie_directors_copy.to_csv("merged-movie.csv")

In [190]:
# check
merged_frame = pd.read_csv('savelist.csv')
merged_frame

,Unnamed: 0,title,director,director gender
0,0,Sankofa,Haile Gerima,male
1,1,The Starling,Theodore Melfi,male
2,2,Confessions of an Invisible Girl,no_data,no_data
3,3,Intrusion,no_data,no_data
4,4,Avvai Shanmughi,K. S. Ravikumar,male
...,...,...,...,...
6830,6830,10 Things I Hate About You,Gil Junger,male
6831,6831,The Man in the Hat,Stephen Warbeck,male
6832,6832,River,Shanice Mollen-Picard,female
6833,6833,Outpost,Chuck Jones,male


# check directors


In [209]:
df_movies = df_movie_directors_copy.copy() #copy 

In [210]:
len(pd.concat(g for _, g in df_movies.groupby("title") if len(g) > 1))

22

In [211]:
pd.concat(g for _, g in df_movies.groupby("title") if len(g) > 1)

,title,director,director gender,distributor,id,rating score
2410,Abhinetri,Subodh Mukherjee,male,no_data,tt0178218,no_data
5692,Abhinetri,Satish Pradhan,male,no_data,tt0178218,no_data
322,Bliss,Abdullah Oğuz,male,Amazon Studios,tt10333426,4.9/10
6057,Bliss,Doris Dörrie,female,Amazon Studios,tt10333426,4.9/10
924,Boss,R. Raghuraj,male,Netflix,tt2571140,3.9/10
4464,Boss,V. N. Aditya,male,Netflix,tt2571140,3.9/10
6774,Jack,Francis Ford Coppola,male,"(Netflix, InterCom)",tt0116669,4.4/10
6827,Jack,Elisabeth Scharang,female,"(Netflix, InterCom)",tt0116669,4.4/10
1594,Lionheart,Genevieve Nnaji,female,YouTube,tt7707314,100%
5071,Lionheart,Sheldon Lettich,male,YouTube,tt7707314,100%


In [212]:
df_movies.query('title=="Abhinetri"')

,title,director,director gender,distributor,id,rating score
2410,Abhinetri,Subodh Mukherjee,male,no_data,tt0178218,no_data
5692,Abhinetri,Satish Pradhan,male,no_data,tt0178218,no_data


In [194]:
df_newdata = pd.read_csv('new_data.csv')
df_newdata.head(5)

,Unnamed: 0,type,title,director,cast,country,date_added_netflix,release_year,rating,duration,listed_in,description,netflix,amazon,date_added_amazon
0,2,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",No Data,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,1,0,1000
1,5,TV Show,Midnight Mass,Mike Flanagan,"Kate Siegel, Zach Gilford, Hamish Linklater, H...",No Data,"September 24, 2021",2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brin...,1,0,1000
2,6,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",No Data,"September 24, 2021",2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...,1,0,1000
3,7,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","September 24, 2021",1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s...",1,0,1000
4,8,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,"September 24, 2021",2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...,1,0,1000


In [213]:
df_newdata.query('title=="Abhinetri"')

,Unnamed: 0,type,title,director,cast,country,date_added_netflix,release_year,rating,duration,listed_in,description,netflix,amazon,date_added_amazon
3729,6078,Movie,Abhinetri,A. L. Vijay,"Tamannaah Bhatia, Prabhu Deva, Sonu Sood, Sapt...",India,"May 1, 2018",2016,TV-14,131 min,"Comedies, International Movies, Sci-Fi & Fantasy","Due to family pressure, a corporate man reluct...",1,1,1000
9943,6895,Movie,Abhinetri,A.L. Vijay,"Tamannaah Bhatia, Prabhu Deva",No Data,1000,2016,13+,131 min,Horror,"Abhinetri is a story of Krishna, an executive ...",1,1,1000


In [214]:
net_mazon_autors = df_newdata['director'].unique().tolist() #list of unique autors
net_mazon_autors[0:9]
print(len(net_mazon_autors))

8844


In [222]:
net_mazon_titles = df_newdata['title'].unique().tolist() #list of unique autors
net_mazon_titles[0:9]
print(net_mazon_titles)

['Ganglands', 'Midnight Mass', 'My Little Pony: A New Generation', 'Sankofa', 'The Great British Baking Show', 'The Starling', 'Bangkok Breaking', 'Je Suis Karl', 'Confessions of an Invisible Girl', 'Intrusion', 'Avvai Shanmughi', 'Go! Go! Cory Carson: Chrissy Takes the Wheel', 'Jeans', 'Minsara Kanavu', 'Grown Ups', 'Dark Skies', 'Paranoia', 'Ankahi Kahaniya', 'The Father Who Moves Mountains', 'The Stronghold', 'Birth of the Dragon', 'Jaws', 'Jaws 2', 'Jaws 3', 'Jaws: The Revenge', 'Safe House', 'The Smart Money Woman', 'Training Day', 'InuYasha the Movie 2: The Castle Beyond the Looking Glass', 'InuYasha the Movie 3: Swords of an Honorable Ruler', 'InuYasha the Movie 4: Fire on the Mystic Island', 'InuYasha the Movie: Affections Touching Across Time', 'Naruto Shippuden the Movie: Blood Prison', 'Naruto Shippûden the Movie: Bonds', 'Naruto Shippûden the Movie: The Will of Fire', 'Naruto Shippuden: The Movie', 'Naruto Shippuden: The Movie: The Lost Tower', 'Naruto the Movie 2: Legend o

In [223]:
dir_title = []#facciamo touple director/title prendendo i titoli da wikidata e i director del dataset netflix/amazon
#unique_titles = df_movies['title'].unique()

print(len(net_mazon_titles))

for  index, title in df_movies['title'].iteritems():
    
    director = df_movies['director'][index]
    
    if title in net_mazon_titles:
        
        dir_title.append((title, director))

print(len(dir_title)) # aumenta perchè nel dataset netflix/amazon abbiamo diversi film con stesso titolo ma più autori
print(dir_title[0:7])
   


12100
6685
[('Sankofa', 'Haile Gerima'), ('The Starling', 'Theodore Melfi'), ('Confessions of an Invisible Girl', 'no_data'), ('Intrusion', 'no_data'), ('Avvai Shanmughi', 'K. S. Ravikumar'), ('Jeans', 'Nicolette Krebitz'), ('Minsara Kanavu', 'Rajiv Menon')]


In [226]:
for i in dir_title:
    if i[0] == "Abhinetri":
        print(i)

('Abhinetri', 'Subodh Mukherjee')
('Abhinetri', 'Satish Pradhan')


In [227]:
check_directors = []
index_to_drop =[] #ripuliamo il dataframe di wikidata da tutti i film con autori che non matchano
for  index_n, title in df_movies['title'].iteritems():
    director = df_movies['director'][index_n]
    for tupla in dir_title:
        if tupla[0] == title and tupla[1] != director and director != "no_data" :

            index_to_drop.append(index_n)
            check_directors.append((title, tupla[1], director))

    
df_movie_directors = df_movies.drop(index_to_drop)
        
        
print(len(index_to_drop))
    

22


In [230]:
print(check_directors)
print(len(check_directors))

[('Bliss', 'Doris Dörrie', 'Abdullah Oğuz'), ('Boss', 'V. N. Aditya', 'R. Raghuraj'), ('Skin', 'Hanro Smitsman', 'Afraa Batous'), ('Lionheart', 'Sheldon Lettich', 'Genevieve Nnaji'), ('The Outsider', 'Alf Brustellin', 'Béla Tarr'), ('Sahara', 'Zoltan Korda', 'Ineke Houtman'), ('Abhinetri', 'Satish Pradhan', 'Subodh Mukherjee'), ('Money', 'Doris Dörrie', 'Steven Hilliard Stern'), ('The Competition', 'Claire Simon', 'Joel Oliansky'), ('The Outsider', 'Béla Tarr', 'Alf Brustellin'), ('The Competition', 'Joel Oliansky', 'Claire Simon'), ('Boss', 'R. Raghuraj', 'V. N. Aditya'), ('Skin', 'Afraa Batous', 'Hanro Smitsman'), ('Lionheart', 'Genevieve Nnaji', 'Sheldon Lettich'), ('Abhinetri', 'Subodh Mukherjee', 'Satish Pradhan'), ('Bliss', 'Abdullah Oğuz', 'Doris Dörrie'), ('Sahara', 'Ineke Houtman', 'Zoltan Korda'), ('Money', 'Steven Hilliard Stern', 'Doris Dörrie'), ('Snow White', 'Michael Berz', 'J. Searle Dawley'), ('Jack', 'Elisabeth Scharang', 'Francis Ford Coppola'), ('Snow White', 'J. Se

In [233]:
df_movie_directors.head(5)
len(df_movie_directors)

6666

In [232]:
pd.concat(g for _, g in df_movie_directors.groupby("title") if len(g) > 1)

ValueError: No objects to concatenate

# report

In [234]:
df_movie_report =  df_movie_directors.replace('no_data', np.nan)

In [235]:
df_movie_report.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6666 entries, 0 to 6781
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            6666 non-null   object
 1   director         5740 non-null   object
 2   director gender  5641 non-null   object
 3   distributor      4615 non-null   object
 4   id               6405 non-null   object
 5   rating score     3061 non-null   object
dtypes: object(6)
memory usage: 364.5+ KB


In [236]:
import pandas_profiling as pp

In [237]:
pp.ProfileReport(df_movie_report, minimal = True)

Render HTML: 100%|██████████| 1/1 [00:01<00:00,  1.01s/it]
